In [24]:
import numpy as np
import pandas as pd
import matplotlib as plt

%matplotlib inline

In [28]:
fba = pd.read_csv('../result/fba_abs.csv', index_col=0)

In [36]:
for i in fba.columns:
    for l in fba.index:
        if fba[i][l] == 0:
            fba = fba.drop(l)

In [37]:
fba

,Control,Clone A,Clone B
BIOMASS,0.001803,0.003173,0.002333
R00200,-0.014733,-0.071747,-0.038218
R00268,0.002674,0.004707,0.003461
R00341,-0.003814,-0.006713,-0.004936
R00342,0.000648,0.001140,0.000838
R00351,-0.002674,-0.004707,-0.003461
R00658,0.018547,0.078460,0.043154
R00703,-0.011147,-0.065436,-0.033577
R01015,-0.010194,-0.040849,-0.022768
R01049,0.000232,0.000408,0.000300


In [40]:
t = (fba.T/fba['Control']).T

In [67]:
c = t.sort_index()

In [70]:
t['Clon']

,Control,Clone A,Clone B
BIOMASS,1.0,1.760117,1.294255
R00200,1.0,4.869724,2.594007
R00268,1.0,1.760117,1.294255
R00341,1.0,1.760117,1.294255
R00342,1.0,1.760117,1.294255
R00351,1.0,1.760117,1.294255
R00658,1.0,4.230314,2.326747
R00703,1.0,5.870009,3.012105
R01015,1.0,4.007345,2.233550
R01049,1.0,1.760117,1.294255


In [64]:
for i in t.sort_index():
    print(i, t['Control'][i])

KeyError: 'Control'

In [77]:
for i in t[np.abs(t['Clone A']) > 2].index:
    print(i, '#4682b4')

R00200 #4682b4
R00658 #4682b4
R00703 #4682b4
R01015 #4682b4
R01061 #4682b4
R01070 #4682b4
R01512 #4682b4
R01518 #4682b4
R01786 #4682b4
R02740 #4682b4
R04779 #4682b4
Glucose_input #4682b4
Lactate_output #4682b4
ADP_Exchange #4682b4


In [76]:
for i in sorted(list(set(t.index)-set(t[np.abs(t['Clone A']) > 2].index))):
    print(i, '#b0c4de')

AMP_Exchange #b0c4de
Ammonia_input #b0c4de
BIOMASS #b0c4de
CO2_output #b0c4de
NADP_Exchange #b0c4de
Oxophos #b0c4de
Oxygen_input #b0c4de
R00268 #b0c4de
R00341 #b0c4de
R00342 #b0c4de
R00351 #b0c4de
R01049 #b0c4de
R01056 #b0c4de
R01082 #b0c4de
R01196 #b0c4de
R01325 #b0c4de
R01529 #b0c4de
R01641 #b0c4de
R01830 #b0c4de
R01899 #b0c4de
R01900 #b0c4de
R08575 #b0c4de
acetyl_CoA_hydrolase #b0c4de
